
# Bu soruya cevap arıyoruz: Eğitime geçmeden önce “doğru teknikleri seçtim mi?” önseziyle anlaşılır mı?

Bir projede model eğitmeye başlamadan önce şu iki şey arasında kalırsın:

- **Önsezi / sezgi:** “Bence şu teknik işe yarar…”
- **Gerçeklik:** “Ama bunu training olmadan nasıl bileceğim?”

Bu notebook’un hedefi şu:

1) “Salt önseziyle” doğru/yanlış tespit edilebilir mi?  
2) Eğer edilebiliyorsa, **hangi sinyallerle**?  
3) Eğitim yapmadan önce **hangi kontroller** eksikleri yakalar?  
4) “Kör deneme-yanılma” yerine **pre-flight checklist** nasıl kurulur?  
5) Minimum eğitimle (çok kısa run) en hızlı doğrulama nasıl yapılır?

Bu bakış açısı özellikle CV/YOLO gibi projelerde seni “boşa eğitim”den kurtarır.



---

## 1) Net cevap: Önsezi tek başına yetmez ama “ön tahmin” için kullanılabilir

- Eğitim yapmadan “kesin” karar veremezsin.
- Ama doğru tasarlanmış **ön kontroller** ile:
  - %60–80 oranında doğru yönde karar verebilirsin
  - en büyük hataları erken yakalarsın
  - yanlış pipeline ile 2 gün eğitim yapıp çöp olmaktan kurtulursun

Bu, sezgiyi “ölçülebilir sinyallere” bağlamak demektir.



---

## 2) Eğitimden önce yakalanabilen hatalar (çok kritik)

### 2.1) Veri / etiket problemleri (en sık)
- Yanlış label formatı
- Bounding box taşmaları (negatif, görüntü dışı)
- Çok fazla “missing label”
- Sınıf id mapping hatası
- Duplicate/near-duplicate data leakage (train-val karışması)
- Aspect ratio/resize pipeline hataları (box scaling bozulması)

### 2.2) Task-metric uyumsuzluğu
- mAP_small kritik ama sen yüksek stride backbone seçmişsin
- Segmentation istiyorsun ama mask pipeline yok
- Veri düşük ışık/blur ama augment yok

### 2.3) Compute bütçesi uyumsuzluğu
- Model aşırı büyük → batch çok küçülür → BN bozulur
- VRAM yetmiyor → yanlış optimization

### 2.4) “Aşırı karmaşık” mimari seçimi
- Veri küçük + model dev → overfit
- Dataset gürültülü + aşırı attention → gürültüyü öğrenir



---

## 3) Sezginin dayandığı sinyaller: “Pre-flight” veri analizi

Eğitimden önce çıkarılması gereken en temel istatistikler:

1) **Örnek sayısı / sınıf dağılımı**
2) **Nesne boyutu dağılımı** (detection için en kritiği)
3) **Aspect ratio dağılımı**
4) **Ortalama/STD, parlaklık histogramı** (low-light/noise sinyali)
5) **Blur/noise ölçüleri** (Laplacian variance gibi basit metriklerle)
6) **Duplicate/leakage kontrolü** (hash / perceptual hash)

Bu istatistikler sana şunu söyler:
- “Bu problem small-object mı?”
- “Domain shift var mı?”
- “Augment ihtiyacı ne kadar?”
- “Model kapasitesi yeterli mi fazla mı?”


In [1]:

# Örnek: bir dataset için "pre-flight" metrikleri listesi (template)

preflight_metrics = {
    "data": [
        "num_images, num_labels",
        "class_counts",
        "train/val leakage check (hash/pHash)",
        "resolution distribution (H,W)",
        "brightness/contrast histogram",
        "blur score (variance of Laplacian)",
        "noise proxy (high-frequency energy)",
    ],
    "detection_specific": [
        "bbox area distribution",
        "bbox width/height distribution",
        "small/medium/large ratio",
        "aspect ratio distribution of boxes",
        "crowd/overlap stats (IoU among boxes)",
    ],
    "pipeline": [
        "resize/letterbox correctness (bbox transform test)",
        "augmentation sanity (bbox stays valid?)",
        "normalization correctness (mean/std ranges)",
    ],
    "compute": [
        "expected batch size",
        "BN vs GN decision",
        "fps/latency target",
    ]
}

for k, v in preflight_metrics.items():
    print("\n==", k, "==")
    for item in v:
        print("-", item)



== data ==
- num_images, num_labels
- class_counts
- train/val leakage check (hash/pHash)
- resolution distribution (H,W)
- brightness/contrast histogram
- blur score (variance of Laplacian)
- noise proxy (high-frequency energy)

== detection_specific ==
- bbox area distribution
- bbox width/height distribution
- small/medium/large ratio
- aspect ratio distribution of boxes
- crowd/overlap stats (IoU among boxes)

== pipeline ==
- resize/letterbox correctness (bbox transform test)
- augmentation sanity (bbox stays valid?)
- normalization correctness (mean/std ranges)

== compute ==
- expected batch size
- BN vs GN decision
- fps/latency target



---

## 4) Eğitim yapmadan önce teknik “doğru mu?” sorusunu cevaplayan testler

### 4.1) Sanity check seti (en küçük ama en etkili)

✅ **1 batch overfit testi**  
- Amaç: model 1 batch’i 50–200 iterasyonda ezberleyebiliyor mu?
- Yapamazsa: pipeline/label/forward/loss bozuk.

✅ **Augmentation toggle testi**  
- Augment açık/kapalı kısa run
- Eğer augment kapalıyken bile öğrenmiyorsa sorun daha temel.

✅ **Gradient check (patlıyor mu?)**  
- loss nan/inf oluyor mu?
- grad norm aşırı mı?

✅ **Data pipeline görsel doğrulama**  
- 50 örneği çiz: bbox doğru mu? mask doğru mu?

Bu testler “tam eğitim” değil; 5–20 dakika.

> Önseziyi “hızlı doğrulama” ile birleştirince artık rastgele deneme olmuyor.



---

## 5) “Eksik nerede?” sorusu için hızlı arıza teşhisi

### 5.1) 1 batch overfit başarısızsa
- label mapping
- loss implementasyonu
- bbox transform (resize/letterbox)
- lr çok düşük/çok yüksek
- normalize yanlış (input range bozuk)

### 5.2) Overfit oluyor ama validation kötü
- data leakage yoksa → genelleme sorunu:
  - augment artır
  - regularization (dropblock/droppath) düşün
  - model küçült
  - label noise temizliği

### 5.3) Küçük nesne kaçırıyorsa (train’den önce bile tahmin edilebilir)
- bbox area dağılımı small ağırlıklıysa:
  - P3 stride 8 kritik
  - yüksek çözünürlük, uygun augment
  - neck fusion iyileştirme



---

## 6) “Önsezi” nasıl geliştirilir? (sistematik yöntem)

Önsezi = rastgele his değil.  
Aşağıdaki 3 kaynaktan beslenir:

1) **Benzer problem bankası**  
   - geçmiş projeler, paper’lar, repo’lar
   - “benim data şuna benziyor” diyebilmek

2) **Hata tipleri → çözüm eşleştirme**  
   - small-object FN → stride/neck/augment
   - FP → calibration/augment/data cleaning
   - localization → loss/assign

3) **Maliyet bilinci (Pareto)**  
   - +0.5 mAP için 2× latency isteniyor mu?
   - hedef gerçek zaman mı, offline mı?

Bu üçü oturduğunda “eğitimden önce” bile doğru hamleleri seçmeye başlarsın.



---

## 7) Sonuç: Kısa ve net

- Salt önseziyle “doğru mu?” kesin bilinmez.
- Ama eğitimden önce yapılan **pre-flight checklist** + **çok kısa sanity run** ile
  büyük hataların çoğu yakalanır.
- Böylece sezgi = ölçülebilir sinyallerle beslenen bir karar mekanizmasına dönüşür.

Aşağıdaki checklist’i projene direkt yapıştırabilirsin.



### Pre-flight Checklist (copy-paste)

**Data**
- [ ] Train/val leakage (hash/pHash)
- [ ] Label mapping doğru
- [ ] BBox/mask valid (bounds, negative yok)
- [ ] Class distribution ve imbalance raporu
- [ ] Small/medium/large oranı raporu

**Pipeline**
- [ ] Resize/letterbox dönüşümü test edildi
- [ ] Augment sonrası bbox geçerli
- [ ] Normalization range doğru (0-1 / mean-std)

**Model**
- [ ] Batch size → BN/GN kararı
- [ ] Output shape ve loss match
- [ ] Params/FLOPs budget uyumlu

**Sanity Runs**
- [ ] 1 batch overfit (50–200 iter)
- [ ] Loss NaN/Inf yok
- [ ] Grad norm patlamıyor
